# go-monkey 動作確認

[Go言語でつくるインタプリタ](https://www.oreilly.co.jp/books/9784873118222/)で実装したMonkey言語インタプリタの動作確認ノートブック

In [8]:
// ローカルのmonkeyモジュールをGoNBの一時コンパイルディレクトリに登録
!cd "$GONB_TMP_DIR" && go mod edit -require=monkey@v0.0.0 -replace=monkey=/Users/junhat6/ghq/github.com/junhat6/go-monkey/monkey

In [9]:
import (
    "fmt"
    "monkey/token"
    "monkey/lexer"
    "monkey/parser"
    "monkey/evaluator"
    "monkey/object"
)

// Monkeyコードを実行して結果を表示するヘルパー
func runMonkey(input string) {
    l := lexer.New(input)
    p := parser.New(l)
    program := p.ParseProgram()
    if errs := p.Errors(); len(errs) > 0 {
        fmt.Println("parse error:", errs)
        return
    }
    env := object.NewEnvironment()
    macroEnv := object.NewEnvironment()
    evaluator.DefineMacros(program, macroEnv)
    expanded := evaluator.ExpandMacros(program, macroEnv)
    result := evaluator.Eval(expanded, env)
    if result != nil {
        fmt.Println(result.Inspect())
    }
}

## Lexer（字句解析）

ソースコードをトークン列に分割する

In [10]:
%%
l := lexer.New(`let add = fn(a, b) { a + b };`)
for tok := l.NextToken(); tok.Type != token.EOF; tok = l.NextToken() {
    fmt.Printf("%-10s %q\n", tok.Type, tok.Literal)
}

LET        "let"
IDENT      "add"
=          "="
FUNCTION   "fn"
(          "("
IDENT      "a"
,          ","
IDENT      "b"
)          ")"
{          "{"
IDENT      "a"
+          "+"
IDENT      "b"
}          "}"
;          ";"


## Parser（構文解析）

トークン列からAST（抽象構文木）を生成する

In [11]:
%%
l := lexer.New(`let x = (5 + 10) * 2;`)
p := parser.New(l)
program := p.ParseProgram()
fmt.Println(program.String())

let x = ((5 + 10) * 2);


## 式の評価

整数演算、比較、文字列結合

In [12]:
%%
// 整数演算
runMonkey(`5 + 10 * 2`)
runMonkey(`(5 + 10) * 2`)

// 比較・真偽値
runMonkey(`1 < 2`)
runMonkey(`10 == 10`)
runMonkey(`!true`)

// 文字列結合
runMonkey(`"Hello" + " " + "World!"`)

25
30
true
true
false
Hello World!


## 変数・関数・クロージャ

In [13]:
%%
// 変数束縛
runMonkey(`let x = 5; let y = 10; x + y;`)

// if/else式（値を返す）
runMonkey(`if (10 > 5) { "yes" } else { "no" }`)

// 関数
runMonkey(`let add = fn(a, b) { a + b }; add(3, 4);`)

// クロージャ
runMonkey(`let newAdder = fn(x) { fn(y) { x + y } }; let addTwo = newAdder(2); addTwo(5);`)

// 高階関数
runMonkey(`let apply = fn(f, x) { f(x) }; apply(fn(x) { x * 2 }, 10);`)

15
yes
7
7
20


## データ構造と組み込み関数

In [14]:
%%
// 配列
runMonkey(`let a = [1, 2, 3, 4, 5]; a[2];`)
runMonkey(`len([1, 2, 3])`)
runMonkey(`first([1, 2, 3])`)
runMonkey(`last([1, 2, 3])`)
runMonkey(`rest([1, 2, 3])`)
runMonkey(`push([1, 2, 3], 4)`)

// ハッシュ
runMonkey(`let h = {"name": "Monkey", "version": 1}; h["name"];`)

// 文字列の長さ
runMonkey(`len("Hello")`)

3
3
1
3
[2, 3]
[1, 2, 3, 4]
Monkey
5


## エラーハンドリング

型の不一致などはエラーオブジェクトとして伝播する

In [15]:
%%
runMonkey(`5 + true`)
runMonkey(`-true`)

ERROR: type mismatch: INTEGER + BOOLEAN
ERROR: unknown operator: -BOOLEAN


## マクロシステム

`quote`/`unquote` でASTを操作し、`macro` で構文を拡張できる

In [16]:
%%
// unlessマクロ: conditionがfalseのときconsequenceを評価する
runMonkey(`
let unless = macro(condition, consequence, alternative) {
    quote(if (!(unquote(condition))) {
        unquote(consequence);
    } else {
        unquote(alternative);
    });
};
unless(10 > 5, "not greater", "greater");
`)

greater
